In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
abbr = 'TN'

In [4]:
file = 'tennessee2018.xlsx'

Read in federal level data

In [5]:
fiscal = pd.read_sas('../../data/fiscal2018', format = 'sas7bdat', encoding='iso-8859-1')

Generate list of districts in the state in the federal data

In [66]:
state_fiscal = fiscal[(fiscal['STABBR'] == abbr) & (fiscal['GSHI'] == '12')]

In [67]:
len(state_fiscal)

129

Read in state level data

In [47]:
state_grads = pd.read_excel('../../data/state_data_raw/' + file, sheet_name='grad_rate')

In [48]:
state_grads.head()

,district_number,district_name,school_number,school_name,year,subgroup,denominator,grad_rate,grad_rate_previous,baseline_year
0,0,State of Tennessee,0,NaN,2017,All Students,71505.0,89.1,88.5,2016.0
1,0,State of Tennessee,0,NaN,2017,Black/Hispanic/Native,22991.0,83.0,82.6,2016.0
2,0,State of Tennessee,0,NaN,2017,Economically Disadvantaged,23539.0,83.7,85.5,2016.0
3,0,State of Tennessee,0,NaN,2017,English Learners,2180.0,73.3,75.6,2016.0
4,0,State of Tennessee,0,NaN,2017,Foster,0.0,*,*,NaN


Reset columns.

In [10]:
# state_grads.columns = state_grads.loc[0]
# state_grads = state_grads.loc[1:]

Filter results.

In [49]:
state_grads = state_grads[pd.isna(state_grads['school_name']) &
                         (state_grads['subgroup'] == 'All Students')]

In [50]:
state_grads

,district_number,district_name,school_number,school_name,year,subgroup,denominator,grad_rate,grad_rate_previous,baseline_year
0,0,State of Tennessee,0,NaN,2017,All Students,71505.0,89.1,88.5,2016.0
15,10,Anderson County Schools,0,NaN,2017,All Students,527.0,93.7,95.5,2016.0
272,11,Clinton City School District,0,NaN,2017,All Students,NaN,*,*,NaN
332,12,Oak Ridge City Schools,0,NaN,2017,All Students,332.0,89.5,88.3,2016.0
453,20,Bedford County Schools,0,NaN,2017,All Students,571.0,90.9,92.1,2016.0
...,...,...,...,...,...,...,...,...,...,...
28168,961,Alvin C York Institute,0,NaN,2017,All Students,143.0,93.0,92.9,2016.0
28199,963,Tennessee School for the Blind,0,NaN,2017,All Students,14.0,14.3,64.3,2016.0
28230,964,Tennessee School for the Deaf,0,NaN,2017,All Students,22.0,72.7,78.9,2016.0
28290,985,Achievement School District,0,NaN,2017,All Students,867.0,53.2,40.4,2016.0


Select and rename columns.

In [51]:
state_grads = state_grads[['district_name', 
                           'denominator',
                           'grad_rate']]

In [52]:
state_grads.columns = ['District Name', 'Total', 'Graduation Rate']

In [53]:
state_grads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148 entries, 0 to 28791
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   District Name    147 non-null    object 
 1   Total            132 non-null    float64
 2   Graduation Rate  148 non-null    object 
dtypes: float64(1), object(2)
memory usage: 4.6+ KB


Convert data types.

In [54]:
state_grads.fillna('', inplace=True)

In [55]:
state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('\*', '')

In [56]:
state_grads['Graduation Rate'] = pd.to_numeric(state_grads['Graduation Rate']) / 100

Check for matches and non-matches in the two lists. 

In [19]:
# state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.upper().str.strip()
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.upper().str.strip()

In [97]:
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' City', '')
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Schools', '')
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' School District', '')
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' School System', '')
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Municipal', '')
state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(' Special', '')

state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' City', '')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' Schools', '')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' School District', '')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' Municipal', '')

In [98]:
matches = [name for name in list(state_grads['District Name']) if name in list(state_fiscal['NAME'])]
matches.sort()
len(matches)

115

In [99]:
A = [name for name in list(state_grads['District Name']) if name not in list(state_fiscal['NAME'])]
A.sort()
A

['',
 'Alamo',
 'Athens',
 'Bells',
 'Benton',
 'Clarksville-Montgomery County',
 'Clinton',
 'Dayton School',
 'Etowah School',
 'Fayette County',
 'Franklin',
 'Gibson County',
 'Hollow Rock-Bruceton',
 'Huntingdon',
 'Jackson-Madison County',
 'Lakeland',
 'Lebanon',
 'Lexington',
 'Lincoln County Department of Education',
 'Manchester',
 'Metro Nashville Public',
 'Murfreesboro',
 'Newport School',
 'Paris',
 'Richard Hardy Memorial School',
 'Rogersville School',
 'South Carroll County',
 'State of Tennessee',
 'Sweetwater',
 'Tennessee School for the Blind',
 'Tennessee State Board of Education',
 'West Carroll',
 'West Tennessee School for the Deaf']

In [100]:
B = [name for name in list(state_fiscal['NAME']) if name not in list(state_grads['District Name'])]
B.sort()
B

['Benton County',
 'Carroll County',
 'Davidson County',
 'Fayette County Public',
 'Gibson Co Sp Dist',
 'Hollow Rock - Bruceton',
 'Huntingdon Special',
 'Lincoln County',
 'Madison County',
 'Montgomery County',
 'Richard',
 'South Carroll',
 'Tennessee School for Blind',
 'West Carroll Sp Dist']

Make any additional matches I can find.

In [24]:
state_fiscal_rename = {
    'Benton County' : 'Benton',
    'Gibson Co Sp Dist' : 'Gibson County',
    'Hollow Rock - Bruceton',
    'Huntingdon Special',
    'Lincoln County',
    'Madison County',
    'Montgomery County',
    'Richard',
    'South Carroll',
    'Tennessee School for Blind',
    'West Carroll Sp Dist'
}

In [25]:
state_fiscal = state_fiscal.replace(state_fiscal_rename)

Merge federal and state data, keeping only matches between the two. 

In [26]:
state_grads_merged = pd.merge(state_fiscal, state_grads, how='inner', left_on='NAME', right_on='District Name')

Save cleaned data. 

In [27]:
state_grads_merged.to_csv('../../data/state_data_merged/' + abbr + '.csv', index=False)